Alan Braeley and Christian Ruiz

In [1]:
# The standard fare:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits 
import astropy.stats as stat

from photutils import aperture_photometry, CircularAperture, CircularAnnulus, DAOStarFinder

# Aperture Photometry with Python Photutils

## 9.1 - Getting Started

The goal of this lab is to teach you how to extract aperture photometry for all of the stars in a given image. Although the extraction itself is automated, there is some art to setting the parameters for extraction, and this lab is designed to teach you to choose these in an intelligent, data-driven way. 

To start we need to read in a raw image to work with. 

In [2]:
#as an example, let's load in a randomly-selected raw Smith R band image
image = fits.getdata('M52-007_R.fit')

We will be using a python package called photutils, which is based on an old IRAF package of the same name. One of the key functions within this package is the DAOStarFinder function, which we'll just call "the star finder" here.

The star finder will extract sources, defined as some multiple (which you provide) of the background/sky level, so first we need to get a reasonable estimate of the background level. This is done using the function mad_std from the astropy.stats package, as below. 

In [3]:
from astropy.stats import mad_std
bkg_sigma = mad_std(image)
print(bkg_sigma)

16.3086244036


In [4]:
print(np.std(image))

313.858966808


### Exercise 1. Use the resources (docstrings, wikipedia, other functions) at your disposal to answer the following questions in the cell below.  
1. What does the mad_std function do?
2. How is it different from the more typical np.std function? How different are the answers in these two cases, and why?  

In [7]:
?mad_std

***Your answers go here***
1. mad_std takes the standard deviation using the "Median Absolute Deviation". MAD is defined as the median of the absolute deviations from the data's median. 

2. The difference between np.std and MAD_std is that the MAD takes an absolute value to be the distance from the "average" (average being mean, median or mode)

## 9.2 - Extracting Stars

The star finder requires two parameters to extract stars in the image:  

1) The threshhold, which we will define as some number of background levels, above which we will call something a star    
2) An estimate for the FWHM of point-sources (stars) in the image  

### Exercise 2
To start, estimate the FWHM of the stars in your image using pyraf's imexam functions, as you did in Lab 8. Measure the FWHM for at least 10 stars and average them. In the cell below, paste the imexam output and calculate the average of the measurements in the cell below that. Insert your calculated average FWHM in the third cell below.  

***insert imexam output here***

In [8]:
#insert calculation of average FWHM here
star1=7.82
star2=8.13
star3=8.33
star4=9.23
star5=9.25
star6=7.94
star7=7.9
star8=8.7
star9=8.18
star10=7.94

mean = (star1+star2+star3+star4+star5+star6+star7+star8+star9+star10)/10
print(mean)

8.341999999999999


In [10]:
#FWHM= this is a placeholder. INSERT YOUR VALUE IN PLACE OF 10 BELOW
FWHM=8.341999999999999

We also need to set the threshhold (described above) for star finder, which we define as some multiple (nsigma) of the background level. To start, let's set nsigma to 10, meaning that in order for somehting to be considered a star, it must have at least 10x the detector counts of the background level. 

The next several lines below set up the parameters for the star finder (by specifying the FWHM and threshhold parameters), apply the star finder to the image, and then extract and print the number of sources.

In [11]:
nsigma=10
daofind = DAOStarFinder(fwhm=FWHM, threshold=nsigma*bkg_sigma)
sources = daofind(image)
nstars=len(sources)
nstars

545

To check how well we're doing here, we need to be able to see how the sources that were automatically extracted line up with apparent sources in the image. To do so, we are going to write the information that star finder extracted from the sources it found into a DS9 region file, so that we can load it with the image. DS9 region files are text files where each line contains the following basic info:  
regiontype xcen ycen FWHM  

The code below writes the relevant outpt from daofind into a text file with this format. 

In [12]:
xpos = np.array(sources['xcentroid'])
ypos = np.array(sources['ycentroid'])
f = open('M52_R.reg', 'w') #you will need to change the first input if you want to write to a different filename later
for i in range(0,len(xpos)):
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(FWHM)+'\n')
f.close()

To display this region file, you should open the science image in DS9, then click Region --> Load Regions and navigate to the .reg file above. When you load it, you will see green circles appear on top of all of the stars that the Star Finder has extracted. Place a screenshot of this overlay in the cell below. 

![IMAGE](first.png)

### Exercise 3 

Using the **median-combined V and R images that you generated for Homework 9**, answer the following questions. Include code, screenshots, etc. to support your argument, and add cells below as needed to do calculations, generate new region files, etc.  
1) How many sources can you extract at V and R for nsigma=10 from your median-combined images? How much does the number of sources vary between the two wavelengths and why, do you think?  
2) How different are the number of extracted stars in the raw R image you used in the example vs. the reduced and median combined R image that you generated for your homework? Name at least one potential reason why they are different, and find an example in the images that shows it.  
*Hint: An example really means a source that was identified in one image and not the other. Remember you can load multiple images in DS9, and can load a separate region file in each. Zoom in on your discrpant source. To match the zoom level and location between the two images, select Frame --> Match --> Frame --> Physical.*  
3) For one of your images (V or R), discuss how the number of extracted sources changes when you change the nsigma threshhold. Make an argument based on the images for what you think the most reasonable limit is for this data.    

#### 1. Problem 1 explanation and images go here

In [15]:
#problem 1 code goes here
image_R = fits.getdata('M52_R_combined.fit')


star1=8.05
star2=9.19
star3=8.65
star4=8
star5=9.32
star6=7.89
star7=9.35
star8=9.19
star9=8.85
star10=8.14

mean = (star1+star2+star3+star4+star5+star6+star7+star8+star9+star10)/10
print(mean)

FWHM_R = mean



nsigma=10
daofind = DAOStarFinder(fwhm=FWHM_R, threshold=nsigma*bkg_sigma)
sources = daofind(image_R)
nstars=len(sources)
print(nstars)


xpos = np.array(sources['xcentroid'])
ypos = np.array(sources['ycentroid'])
f = open('M52_R_R.reg', 'w') #you will need to change the first input if you want to write to a different filename later
for i in range(0,len(xpos)):
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(FWHM)+'\n')
f.close()

8.663


/Users/astrolab/anaconda/lib/python3.6/site-packages/photutils/detection/findstars.py:399: RuntimeWarning: invalid value encountered in greater
  object_labels, nobjects = ndimage.label(convolved_data > threshold,


497


/Users/astrolab/anaconda/lib/python3.6/site-packages/photutils/detection/core.py:241: RuntimeWarning: invalid value encountered in greater
  peak_goodmask = np.logical_and(peak_goodmask, (data > threshold))
/Users/astrolab/anaconda/lib/python3.6/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, oper)(other)
/Users/astrolab/anaconda/lib/python3.6/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)


![IMAGE](second.png)

In [16]:
#problem 1 code goes here
image_V = fits.getdata('M52_V_combined.fit')


star1=9.82
star2=10.05
star3=9.93
star4=9.86
star5=10.29
star6=10.09
star7=10
star8=9.87
star9=10.04
star10=10.02

mean = (star1+star2+star3+star4+star5+star6+star7+star8+star9+star10)/10
print(mean)

FWHM_V = mean



nsigma=10
daofind = DAOStarFinder(fwhm=FWHM_V, threshold=nsigma*bkg_sigma)
sources = daofind(image_V)
nstars=len(sources)
print(nstars)


xpos = np.array(sources['xcentroid'])
ypos = np.array(sources['ycentroid'])
f = open('M52_R_V.reg', 'w') #you will need to change the first input if you want to write to a different filename later
for i in range(0,len(xpos)):
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(FWHM)+'\n')
f.close()

9.996999999999998


/Users/astrolab/anaconda/lib/python3.6/site-packages/photutils/detection/findstars.py:399: RuntimeWarning: invalid value encountered in greater
  object_labels, nobjects = ndimage.label(convolved_data > threshold,


369


/Users/astrolab/anaconda/lib/python3.6/site-packages/photutils/detection/core.py:241: RuntimeWarning: invalid value encountered in greater
  peak_goodmask = np.logical_and(peak_goodmask, (data > threshold))
/Users/astrolab/anaconda/lib/python3.6/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, oper)(other)
/Users/astrolab/anaconda/lib/python3.6/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)


![IMAGE](third.png)

For our R image we extracted 497 stars and for our V image we extracted 369 stars. This is because the R image is a band sensitive to a lower energy and therefore dimmer stars. 

#### 2. Problem 2 explanation and images go here

The number of stars registered in the raw image is 545. The number in the reduced image is 497. This is because the higher FWHM on the reduced image means that less stars (the dimmer ones) will be enough standard deviations above the noise level to actually register as a star.

![IMAGE](fourth.png)

#### 3. Problem 3 explanation and images go here

In [20]:
#problem 3 code goes here
image_R = fits.getdata('M52_R_combined.fit')


star1=8.05
star2=9.19
star3=8.65
star4=8
star5=9.32
star6=7.89
star7=9.35
star8=9.19
star9=8.85
star10=8.14

mean = (star1+star2+star3+star4+star5+star6+star7+star8+star9+star10)/10
print(mean)

FWHM_R = mean



nsigma=2
daofind = DAOStarFinder(fwhm=FWHM_R, threshold=nsigma*bkg_sigma)
sources = daofind(image_R)
nstars=len(sources)
print(nstars)


xpos = np.array(sources['xcentroid'])
ypos = np.array(sources['ycentroid'])
f = open('M52_R_R2.reg', 'w') #you will need to change the first input if you want to write to a different filename later
for i in range(0,len(xpos)):
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(FWHM)+'\n')
f.close()

8.663


/Users/astrolab/anaconda/lib/python3.6/site-packages/photutils/detection/findstars.py:399: RuntimeWarning: invalid value encountered in greater
  object_labels, nobjects = ndimage.label(convolved_data > threshold,
/Users/astrolab/anaconda/lib/python3.6/site-packages/photutils/detection/core.py:241: RuntimeWarning: invalid value encountered in greater
  peak_goodmask = np.logical_and(peak_goodmask, (data > threshold))


1196


/Users/astrolab/anaconda/lib/python3.6/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, oper)(other)
/Users/astrolab/anaconda/lib/python3.6/site-packages/astropy/table/column.py:929: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)


When we make nsigma 1 we register over 4000 sources, many of which are clearly simply products of background noise. When nsigma is 2, we only register approximately 1200 sources. As opposed to nsigma = 1, these all seem to actually be sources, making nsigma =2 a reasonable bottom limit.

![IMAGE](fifth.png)

### 9.3 - Aperture Photometry

The next step is to actually extract the photometry, and here too there is some art to choosing parameters. Although photutils will extract the photometry for each star in an automated way, you need to intelligently choose the parameters based on your data.  

The tunable parameters are:
1. the aperture radius inside of which to count the flux from the star
2. the inner and outer radius of the sky aperture. The annulus defined by these two numbers needs to be large enough to get a good measurement of the background level, but small enough to generally avoid confusion with other nearby sources. 

We'll start with some potentially reasonable values for these parameters. 

In [21]:
aprad = 8
skyin=10
skyout=15

### Exercise 4

For each of the next two cells, write a comment describing what each line is doing in the line above it. 

In [22]:
#add comments to this cell

starapertures = CircularAperture((xpos,ypos),r=aprad)
#This line is setting the aperture that will take in source photons
skyannuli = CircularAnnulus((xpos,ypos),r_in=skyin,r_out=skyout)
#This one sets the annuli which determines the sky background
phot_apers = [starapertures, skyannuli]
#This creates an array with the values for the star-detecting aperture and the background annuli in their own respective elements
phot_table = aperture_photometry(image,phot_apers)
#This just creates a table and then assigns each block a value based on the aperture or annuli positions.
phot_table
#This plots the table

id,xcenter,ycenter,aperture_sum_0,aperture_sum_1
,pix,pix,,
int64,float64,float64,float64,float64
1,1598.819318973744,34.02767752829353,242885.241427,474118.456593
2,3013.42982130593,34.09719382856377,241592.644525,472247.864067
3,900.27138272713,35.1241483459529,242765.167691,474165.730973
4,2435.9288680400305,35.36475903809039,243937.451595,473672.719616
5,2662.481761177369,37.3244327140168,242143.414382,472680.700176
6,2813.944709726853,43.584373842088226,242063.865496,473844.974098
7,1835.077213421611,45.88848965716383,242812.396109,474212.291408
8,1498.6011844785041,49.006504892020686,242743.552914,474105.081909


In [23]:
#add comments to this cell
bkg_mean = phot_table['aperture_sum_1']/skyannuli.area()
#This gets the mean average of the background from the annulus
bkg_starap_sum = bkg_mean * starapertures.area()
#This is the background noise that is predicted inside of the star-detecting aperture
final_sum = phot_table['aperture_sum_0']-bkg_starap_sum
#This subtracts the background from the star-detecting aperture
phot_table['background subtracted star counts'] = final_sum
#This assigns the background-subtracted star-detecting aperture from what is detected inside the star-detecting aperture -> ideally giving us just source photons
phot_table
#This plots the values

id,xcenter,ycenter,aperture_sum_0,aperture_sum_1,background subtracted star counts
,pix,pix,,,
int64,float64,float64,float64,float64,float64
1,1598.819318973744,34.02767752829353,242885.241427,474118.456593,136.591651161
2,3013.42982130593,34.09719382856377,241592.644525,472247.864067,-198.261877106
3,900.27138272713,35.1241483459529,242765.167691,474165.730973,-7.68656736068
4,2435.9288680400305,35.36475903809039,243937.451595,473672.719616,1417.01915125
5,2662.481761177369,37.3244327140168,242143.414382,472680.700176,130.895891593
6,2813.944709726853,43.584373842088226,242063.865496,473844.974098,-544.761242467
7,1835.077213421611,45.88848965716383,242812.396109,474212.291408,15.7029085674
8,1498.6011844785041,49.006504892020686,242743.552914,474105.081909,1.75097728218


### Exercise 5 -

Spend the rest of the lab time investigating what changes about the photometric measurement (background subtracted sky counts caluclated column) when you adjust the tunable parameters (aperture radius and inner/outer sky annulus) and report your findings below. You may wish to examine only a handful of stars in the table and avoid pr (to print just a few rows of a table, see the example below), but make sure the rows you select include stars with a range of brightnesses. You may also wish to make separate versions/copies of the table with different aperture parameters so that you can compare without overwriting. Think in particular about crowded fields and see if you can derive the best parameters for this case as well (identify things in the table with very close values for xcenter and ycenter to find these crowded regions). 

In [50]:
#example of printing just a few rows in a table - chosen to be one bright star, one faint

aprad = 5
skyin=1
skyout=10

starapertures = CircularAperture((xpos,ypos),r=aprad)
#This line is setting the aperture that will take in source photons
skyannuli = CircularAnnulus((xpos,ypos),r_in=skyin,r_out=skyout)
#This one sets the annuli which determines the sky background
phot_apers = [starapertures, skyannuli]
#This creates an array with the values for the star-detecting aperture and the background annuli in their own respective elements
phot_table = aperture_photometry(image,phot_apers)

bkg_mean = phot_table['aperture_sum_1']/skyannuli.area()
#This gets the mean average of the background from the annulus
bkg_starap_sum = bkg_mean * starapertures.area()
#This is the background noise that is predicted inside of the star-detecting aperture
final_sum = phot_table['aperture_sum_0']-bkg_starap_sum
#This subtracts the background from the star-detecting aperture
phot_table['background subtracted star counts'] = final_sum
#This assigns the background-subtracted star-detecting aperture from what is detected inside the star-detecting aperture -> ideally giving us just source photons
phot_table[6:16]
#This plots the values

id,xcenter,ycenter,aperture_sum_0,aperture_sum_1,background subtracted star counts
,pix,pix,,,
int64,float64,float64,float64,float64,float64
7,1835.077213421611,45.88848965716383,94917.5666713,1042835.49027,114.340282709
8,1498.6011844785041,49.006504892020686,94938.7671384,1042951.88346,124.959551611
9,2448.805643055918,49.447539262412306,94754.4476445,1040900.01169,127.173854416
10,2007.757925486525,51.62678637785862,94601.4197092,1040932.48749,-28.8064265728
11,1562.7151354347313,52.565497369716574,94687.4651249,1058158.84929,-1508.79390132
12,2367.7477519996864,52.90181110415589,94698.2655138,1040451.57171,111.758994381
13,2746.96265698573,53.25758553044239,94533.9460388,1038853.2262,92.7436569777
14,2589.542861238097,54.86357879265668,102123.828457,1896211.50057,-70259.0352305


In order to get the best parameters you need to establish what should be consisten throughout the image: this would be the background (ideally speaking of course). To therefore find this value you want to adjust the inner and outer annuli until the aperture_sum_1 column gives a ralatively consisten number. This would mean that you have reached the best case scenerio with which to adjust the aperture data. 

The aperture data is extremely unpredictable, unlike the background sky. We therefore must be able to trust our value for the background sky (aperture_sum_1). When we subtract the background from the aperture/star values, we get our calculated value for the star in the final column, "background subtracted star counts". We know that if this column has negative values, we need to be more strict with the number of standard deviations with which we require pixels to be before we register them as stars -> i.e. we must increase nsigma. A negative value in the "background subtracted star counts" could also be caused by the value of an even brighter star being subtracted from the aperture radius. This would occur if a star was also located inside the annuli. *However*, this is unlikely to be a problem if our background values/aperture_sum_1's are all the same, or at least approximately consistent. We can therefore safely assume this isn't happening so long as we have a large number of stars being sampled, all with consisten background values.

Once you have the background subtracted from the aperture radius you should be left with the starlight alone.